In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import csv
import time
import random
import requests
from bs4 import BeautifulSoup

options = Options()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--profile-directory=Default")
options.add_argument("--user-data-dir=D:/Data Science/var/tmp/chrome_user_data")

driver = webdriver.Chrome(options=options)

agent_names_numbers = []

try:
    for page_num in range(1, 2):  # Loop through all 188 pages
        url = f"https://www.zameen.com/agents/Karachi-2/?page={page_num}"
        driver.get(url)

        # Wait for the dynamic content to load with a random delay between 10 to 15 seconds
        time.sleep(random.uniform(3, 5))

        # Get all the call buttons
        call_buttons = driver.find_elements(By.XPATH, '//button[contains(@class, "agent-listing-card_propertyCallBtn__3E8nV")]')

        for index, call_button in enumerate(call_buttons):
            # Scroll into view
            ActionChains(driver).move_to_element(call_button).perform()

            # Click the call button
            call_button.click()

            # Wait for the phone number to be visible after the click
            phone_number_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//div[@class="flex flexYcenter u-spbwx12 text-link"]/div'))
            )
            phone_number = phone_number_element.text

            # Find the agent name using BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")
            agent_name = soup.find_all('h3', class_='u-mb4 agent-listing-card_cardTitle__1l-0P heading_h3__yJ0ks')[index].text

            # Save agent name and phone number
            agent_names_numbers.append((agent_name, phone_number))
            print(f"Page {page_num}, Agent {index + 1}: {agent_name}, {phone_number}")

            # Close the overlay by clicking the close button
            close_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "modal_closeIcon__2B0W7"))
            )
            close_button.click()

            # Add a delay after each iteration to not trigger anti-bot mechanisms
            time.sleep(random.uniform(3, 5))

finally:
    driver.quit()

# Save the data into a CSV file
with open('real_estate_agents.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Agent Name', 'Phone Number'])  # Writing header
    for name, number in agent_names_numbers:
        writer.writerow([name, number])  # Writing agent details

print("Data has been written to real_estate_agents.csv")